# Association rule mining 

#### Imports

In [4]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

#### Load data

In [5]:
df = pd.read_csv('..\..\data\MC1\one_hot_encoding.csv', sep = ',')
df_locations = pd.read_parquet("..\..\data/MC1/locations.parquet")

# rename columns of df to df_locations['location']
df.columns = df_locations['location'].values

<>:1: DeprecationWarning: invalid escape sequence '\.'
<>:2: DeprecationWarning: invalid escape sequence '\.'
<>:1: DeprecationWarning: invalid escape sequence '\.'
<>:2: DeprecationWarning: invalid escape sequence '\.'
C:\Users\20182009\AppData\Local\Temp\ipykernel_8604\4178687442.py:1: DeprecationWarning: invalid escape sequence '\.'
  df = pd.read_csv('..\..\data\MC1\one_hot_encoding.csv', sep = ',')
C:\Users\20182009\AppData\Local\Temp\ipykernel_8604\4178687442.py:2: DeprecationWarning: invalid escape sequence '\.'
  df_locations = pd.read_parquet("..\..\data/MC1/locations.parquet")


#### Run the association rule mining algorithm

In [6]:
df_apriori = apriori(df, min_support = 0.2, use_colnames = True, verbose = 1)
df_ar = association_rules(df_apriori, metric = "confidence", min_threshold = 0.6)
df_ar = df_ar.sort_values(by = "consequent support", ascending = False)

c:\Users\20182009\Documents\Master DSAI\y1 q3\2AMV10 visual analytics\code\visual-analytics-dashboard\.venv\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Processing 21 combinations | Sampling itemset size 76


#### Filter the results to get useful rules

In [7]:
df_filter = df_ar.copy()
drops = []
for index, row in df_filter.iterrows():
    for index2, row2 in df_filter.iterrows():
        if row['antecedents'] == row2['consequents'] and row['consequents'] == row2['antecedents']:
            if index2 not in drops:
                drops.append(index2)
                df_filter = df_filter.drop(index2)
    drop_items = ['ranger-stop0', 'ranger-stop2', 'general-gate2']
    for item in drop_items:
        if frozenset({item}) == row['antecedents'].intersection(frozenset({item})) or frozenset({item}) == row['consequents'].intersection(frozenset({item})):
            if index not in drops:
                drops.append(index)
                df_filter = df_filter.drop(index)

df_filter = df_filter.sort_values(by = "consequent support", ascending = False)

#### Print the results and interpret by hand

In [8]:
df_filter.head(len(df_filter))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(entrance3),(general-gate1),0.417896,0.647477,0.253902,0.607572,0.938369,-0.016676,0.898313
13,(general-gate4),(general-gate1),0.465202,0.647477,0.362786,0.779846,1.204438,0.061578,1.601257
124,"(general-gate5, entrance4)",(general-gate1),0.294045,0.647477,0.285493,0.970914,1.499535,0.095105,12.120181
128,"(general-gate7, general-gate5)",(general-gate1),0.211888,0.647477,0.209536,0.988900,1.527313,0.072343,31.759116
129,"(general-gate5, general-gate4)",(general-gate1),0.207398,0.647477,0.207344,0.999742,1.544058,0.073059,1367.789181
130,"(general-gate7, general-gate4)",(general-gate1),0.464668,0.647477,0.362412,0.779938,1.204580,0.061550,1.601925
458,"(general-gate7, general-gate4, general-gate5)",(general-gate1),0.207024,0.647477,0.207024,1.000000,1.544456,0.072981,inf
11,(entrance4),(general-gate1),0.351400,0.647477,0.285493,0.812443,1.254783,0.057969,1.879551
10,(general-gate5),(general-gate1),0.410092,0.647477,0.373637,0.911105,1.407162,0.108112,3.965625
12,(general-gate7),(general-gate1),0.569810,0.647477,0.365138,0.640807,0.989698,-0.003801,0.981430


#### conclusions 
association rule message:
After performing association rule mining on the data, two main routes where revealed in the data. 

route 1:
Entrance 1 - General gate 7 - General gate 4 - General gate 1* - General gate 5

route 2:
Entrance 2 - General gate 1* - Entrance 3

(General gate 1 - Ranger stop 2 - Ranger stop 0 - General gate 2) are viewed as one place in the analysis due to going through one means going through all of them